IMAGES AND VIDEO


In [ ]:
!pip install ultralytics
from ultralytics import YOLO

model = YOLO("/content/best.pt")
metrics = model.val(data="/content/road_boundary_segmentation_dataset.yaml", save=True)
print(metrics)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

FileNotFoundError: 
Dataset '/content/road_boundary_segmentation_dataset.yaml' images not found ⚠️, missing path '/content/Road-Boundary-Detection-8/valid/images'
Note dataset download directory is '/content/datasets'. You can update this in '/root/.config/Ultralytics/settings.json'

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="\xxxP")
project = rf.workspace("roadboundarydetection").project("road-boundary-detection-rhwne")
version = project.version(8)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Road-Boundary-Detection-8 in yolov8:: 100%|██████████| 1210/1210 [00:00<00:00, 5737.06it/s]


In [ ]:
!pip install ultralytics
import cv2
import numpy as np
import os
import time
from ultralytics import YOLO


model_path = "/content/best.pt"
model = YOLO(model_path)

#input directory
input_path = "/content/frame_0001.jpg"


output_folder = "/content/TESTINg/output/"
os.makedirs(output_folder, exist_ok=True)

# Checking if video
if input_path.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
    is_video = True
    cap = cv2.VideoCapture(input_path)

    #video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    #for unique timestamp
    timestamp = int(time.time())  # Unique timestamp
    output_path = os.path.join(output_folder, f"output_video_{timestamp}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    #for video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run inference
        results = model.predict(frame, conf=0.05, imgsz=640)

        if results[0].masks is not None:
            mask = results[0].masks.data[0].cpu().numpy()
            mask = (mask * 255).astype(np.uint8)
            mask = cv2.resize(mask, (width, height))

            # Create transparent blue overlay
            overlay = frame.copy()
            overlay[mask > 128] = [255, 0, 0]  # Blue fill

            # Blend overlay with original frame (Transparency: 0.3)
            frame = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)

            # Bold red boundary edges
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(frame, contours, -1, (0, 0, 255), 3)  # Red boundary

        out.write(frame)

    cap.release()
    out.release()
    print(f"Processed video saved at: {output_path}")
#for image
else:
    is_video = False
    frame = cv2.imread(input_path)
    height, width, _ = frame.shape

    # Run inference
    results = model.predict(frame, conf=0.5, imgsz=640)

    if results[0].masks is not None:
        mask = results[0].masks.data[0].cpu().numpy()
        mask = (mask * 255).astype(np.uint8)
        mask = cv2.resize(mask, (width, height))

        # Create transparent blue overlay
        overlay = frame.copy()
        overlay[mask > 128] = [255, 0, 0]  # Blue fill

        # Blend overlay with original frame (Transparency: 0.3)
        frame = cv2.addWeighted(overlay, 0.1, frame, 0.7, 0)

        # Bold red boundary edges
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(frame, contours, -1, (0, 0, 255), 3)  # Red boundary


    timestamp = int(time.time())
    output_path = os.path.join(output_folder, f"output_image_{timestamp}.jpg")
    cv2.imwrite(output_path, frame)
    print(f"Processed image saved at: {output_path}")



0: 480x640 1 road_boundary, 36.1ms
Speed: 3.6ms preprocess, 36.1ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)
Processed image saved at: /content/TESTINg/output/output_image_1740332858.jpg
